In [12]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from skimage.io import imread, imshow
from skimage.transform import resize
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit

df = pd.read_csv('images_2.csv')
df

,image,label
0,ea7b6656-3f84-4eb3-9099-23e623fc1018,Top
1,ea2ffd4d-9b25-4ca8-9dc2-bd27f1cc59fa,Top
2,3b86d877-2b9e-4c8b-a6a2-1d87513309d0,Shoes
3,5d3a1404-697f-479f-9090-c1ecd0413d27,Bottoms
4,b0c03127-9dfb-4573-8934-1958396937bf,Top
...,...,...
4672,3855ea22-5e7f-411f-b1fa-6db27a676c06,Shoes
4673,dfd4079d-967b-4b3e-8574-fbac11b58103,Bottoms
4674,5379356a-40ee-4890-b416-2336a7d84061,Bottoms
4675,65507fb8-3456-4c15-b53e-d1b03bf71a59,Shoes


In [13]:
df['label'].value_counts()

Top        2916
Bottoms    1156
Shoes       433
Hat         172
Name: label, dtype: int64

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(df.label)
labels

array([3, 3, 2, ..., 0, 2, 0])

In [15]:
images = df.iloc[:, 0]

In [16]:
from sklearn.model_selection import StratifiedShuffleSplit

ss = StratifiedShuffleSplit(n_splits=1, test_size=0.1)
for _, group_index in ss.split(images, labels):
    group_x = images[group_index].to_numpy()
    group_y = labels[group_index]

In [17]:
ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_index, test_index in ss.split(group_x, group_y):
    train_x, test_x = group_x[train_index], group_x[test_index]
    train_y, test_y = group_y[train_index], group_y[test_index]

In [18]:
ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_index, valid_index in ss.split(train_x, train_y):
    train_x, valid_x = train_x[train_index], train_x[valid_index]
    train_y, valid_y = train_y[train_index], train_y[valid_index]

In [19]:
# load images and convert to tensor

import numpy as np
from skimage.io import imread
from skimage.transform import resize

def get_images(path_list):
    train_img = []
    for filename in path_list:
        path = 'images_compressed/' + filename + '.jpg'
        img = imread(path, as_gray=True)
        img = resize(img, (100, 100))
        img /= 255.0
        img = img.astype('float32')
        train_img.append(img)
    return np.asarray(train_img)

In [20]:
train_x_images = get_images(train_x)

In [21]:
print(train_x.shape)
print(train_x_images.shape)

(299,)
(299, 100, 100)


In [22]:
valid_x_images = get_images(valid_x)
test_x_images = get_images(test_x)

In [23]:
import torch

train_x_torch = torch.from_numpy(train_x_images).unsqueeze(dim=1)
valid_x_torch = torch.from_numpy(valid_x_images).unsqueeze(dim=1)
test_x_torch = torch.from_numpy(test_x_images).unsqueeze(dim=1)

In [24]:
train_x_torch.shape

torch.Size([299, 1, 100, 100])

In [25]:
train_y_torch = torch.from_numpy(train_y)
valid_y_torch = torch.from_numpy(valid_y)
test_y_torch = torch.from_numpy(test_y)
train_y_torch.shape

torch.Size([299])

In [31]:
from torch import nn

# need to figure out what the numbers in Conv2d and other parts are / what to use for them

class ConvNN(nn.Module):
    def __init__(self):
        super(ConvNN, self).__init__()
        
        # two layers for this cnn
        self.cnn_layers = nn.Sequential (
            # Defining a 2D convolution layer
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            
            # Defining another 2D convolution layer
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

    
        self.linear_layers = nn.Sequential(
            Linear(32 * 5 * 5, 4)
        )

    
    # Defining the forward pass    
    def forward(self, x):
        output = self.cnn_layers(x)
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output

In [33]:
from torch import optim

model = ConvNN()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
print(model)

ConvNN(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=800, out_features=4, bias=True)
  )
)


In [34]:
# Training loop

epochs = 10

for epoch in range(epochs):
    running_loss = 0.
    for images, labels in trainloader:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('At epoch', epoch, 'loss=', running_loss/len(trainloader))

NameError: name 'trainloader' is not defined

In [26]:
# visualizing images
'''
i = 0
plt.figure(figsize=(10,10))
plt.subplot(221), plt.imshow(train_x[i], cmap='gray')
plt.subplot(222), plt.imshow(train_x[i+25], cmap='gray')
plt.subplot(223), plt.imshow(train_x[i+50], cmap='gray')
plt.subplot(224), plt.imshow(train_x[i+75], cmap='gray')
'''

"\ni = 0\nplt.figure(figsize=(10,10))\nplt.subplot(221), plt.imshow(train_x[i], cmap='gray')\nplt.subplot(222), plt.imshow(train_x[i+25], cmap='gray')\nplt.subplot(223), plt.imshow(train_x[i+50], cmap='gray')\nplt.subplot(224), plt.imshow(train_x[i+75], cmap='gray')\n"